In [11]:
#import libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time
%matplotlib inline

# A. API Functions

In [12]:
def get_data(date):
    """ Query the API for 2000 days historical price data starting from "date". """
    url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&limit=2000&toTs={}".format(date)
    r = requests.get(url)
    ipdata = r.json()
    return ipdata

In [13]:
#This function accesses API for Json files

def get_data_spec(coin, date, time_period):
    """ Query the API for 2000 units historical price data starting from "date". """
    url = "https://min-api.cryptocompare.com/data/{}?fsym={}&tsym=USD&limit=2000&toTs={}".format(time_period, coin, date)
    r = requests.get(url)
    ipdata = r.json()
    return ipdata

In [14]:
#This function creates a properly formatted dataframe from file

def get_df_spec(time_period, coin, from_date, to_date):
    """ Gets news data between two dates. If further apart than query limit then query multiple times. """
    date = to_date
    holder = []
    while date > from_date:
        # Now we use the new function to query specific coins
        data = get_data_spec(coin, date, time_period) 
        holder.append(pd.DataFrame(data['Data']))
        date = data['TimeFrom'] 
    df = pd.concat(holder, axis = 0)
    df = df[df['time']>from_date]
    df['time'] = pd.to_datetime(df['time'], unit='s') 
    df.set_index('time', inplace=True)
    df.sort_index(ascending=False, inplace=True)
    # And just keep the close price, with the column heading as the name of the coin. 
    df.rename(columns={'close':coin}, inplace=True)
    return df[coin]

In [15]:
#returns dataframe of news articles before date specified

def get_df(timestamp):
    df = pd.DataFrame(get_data_simple(timestamp)['Data'])
    return df

#this function takes in a timestamp
#edit the URL to add in filters
#yahoofinance
def get_data_simple(timestamp):
    url = "https://min-api.cryptocompare.com/data/v2/news/?sortOrder=latest&{}&categories=BTC,ETH,USDT,Mining,Market,Fiat,Regulation,Commodity,Technology,Blockchain&excludeCategories=Sponsored&lang=EN".format('lTs='+ str(timestamp)) #formats timestamp into api query
    #url = "https://min-api.cryptocompare.com/data/v2/news/?{}&excludeCategories=Sponsored&lang=EN".format('lTs='+ str(timestamp)) #formats timestamp into api query
    r = requests.get(url) #gets url
    ipdata = r.json() #saves json file
    return ipdata #returns json for df processing

In [16]:
#this function takes in a timestamp
#edit the URL to add in filters
#yahoofinance
#not going to use any filters except for english for last pass

def get_data_simple(timestamp):
    url = "https://min-api.cryptocompare.com/data/v2/news/?{}&lang=EN".format('lTs='+ str(timestamp)) #formats timestamp into api query
    #url = "https://min-api.cryptocompare.com/data/v2/news/?{}&categories=BTC,ETH,USDT,Mining,Market,Fiat,Regulation,Commodity,Technology,Blockchain&excludeCategories=Sponsored&lang=EN".format('lTs='+ str(timestamp)) #formats timestamp into api query
    #url = "https://min-api.cryptocompare.com/data/v2/news/?{}&excludeCategories=Sponsored&lang=EN".format('lTs='+ str(timestamp)) #formats timestamp into api query
    r = requests.get(url) #gets url
    ipdata = r.json() #saves json file
    return ipdata #returns json for df processing

In [26]:
#need to make a loop with time delays to obtain all the data and concat into one large dataframe

def api_get_df(timestamplist):
    df_list=[] #stores all dataframes
    counter = 0
    for timestamp in timestamplist: #testing purposes
        df_list.append(get_df(timestamp))
        #should enter some sort of time delay because of api request limits
        counter = counter +1
        print (counter)
        if counter % 45 == 0:
            print("Taking a Break")
            time.sleep(1) #Taks a 1 second break after 45 links 
    
    #combine dataframes
    df_final = pd.concat(df_list, axis = 0)
    df_final['date_time'] = pd.to_datetime(df_final['published_on'], unit='s') 
    return df_final #.sort_values(by = 'date_time', ascending = False) #return final combined dataframes
        
    

# 1. Gathering API Data

In [18]:
#Gathering the right data is tricky, need to be careful in setting parameters on what data to pull from API
#Also need to specify dates in unix time stamp

In [19]:
#Creating Dataframe with unix timestamps from a dataframe of date time objects
#Unix timestamps are used as an argument for functions and api
datelist = pd.date_range(end = pd.datetime.today(), periods=1160).tolist()
df_dates = pd.DataFrame(datelist, columns = ['Date_Time'])
#converts datetime into timestamps for the api
df_dates['timestamp'] = df_dates.Date_Time.apply(lambda x: int(datetime.timestamp(x)))

In [20]:
df_dates.dtypes #check correct types

Date_Time    datetime64[ns]
timestamp             int64
dtype: object

In [21]:
df_dates.min() #successfully convereted Date_time into timestamp from my birthday to today

Date_Time    2016-06-15 15:15:48.324299
timestamp                    1466018148
dtype: object

In [22]:
#testing API functions on a short list
#short_list = df_dates.timestamp[0:5]

In [28]:
api_test = api_get_df(df_dates.timestamp) #obtain data using short list of unix timestamps

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
Taking a Break
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
Taking a Break
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
Taking a Break
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
Taking a Break
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
Taking a Break
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
2

In [30]:
api_test = api_test.reset_index()
api_test.drop('index', axis = 1, inplace = True) #reset index and drop old index column
api_test

,body,categories,downvotes,guid,id,imageurl,lang,published_on,source,source_info,tags,title,upvotes,url,date_time
0,CORE and CORE Media were birthed out of the co...,Technology,0,http://www.cryptocoremedia.com/old/supernet/it...,14846,https://images.cryptocompare.com/news/coremedi...,EN,1465603200,coremediasupernet,{},SuperNET News,SuperNET Weekly No. 14,0,http://www.cryptocoremedia.com/old/supernet/it...,2016-06-11 00:00:00
1,CORE and CORE Media were birthed out of the co...,Technology,0,http://coremedia.info/supernet/item/260-supern...,89,https://images.cryptocompare.com/news/default/...,EN,1465603200,coremediasupernet,{},SuperNET News,SuperNET Weekly No. 14,0,http://coremedia.info/supernet/item/260-supern...,2016-06-11 00:00:00
2,CORE and CORE Media were birthed out of the co...,LTC|Trading|Technology,0,http://coremedia.info/supernet/item/256-supern...,90,https://images.cryptocompare.com/news/default/...,EN,1464998400,coremediasupernet,{},SuperNET News,SuperNET Weekly No. 13,0,http://coremedia.info/supernet/item/256-supern...,2016-06-04 00:00:00
3,CORE and CORE Media were birthed out of the co...,LTC|Trading|Technology,0,http://www.cryptocoremedia.com/old/supernet/it...,14847,https://images.cryptocompare.com/news/coremedi...,EN,1464998400,coremediasupernet,{},SuperNET News,SuperNET Weekly No. 13,0,http://www.cryptocoremedia.com/old/supernet/it...,2016-06-04 00:00:00
4,"Donec quam felis, ultricies nec, pellentesque ...",Other,0,http://hashmag.mikado-themes.com/?p=221,356653,https://images.cryptocompare.com/news/cryptoin...,EN,1458563036,cryptoinsider,"{'name': 'CryptoInsider', 'lang': 'EN', 'img':...",,Is hearing loss of young people caused by exte...,0,https://cryptoinsider.com/2016/03/21/is-hearin...,2016-03-21 12:23:56
5,"Donec quam felis, ultricies nec, pellentesque ...",Other,0,http://hashmag.mikado-themes.com/?p=219,356654,https://images.cryptocompare.com/news/cryptoin...,EN,1458562973,cryptoinsider,"{'name': 'CryptoInsider', 'lang': 'EN', 'img':...",,Read this tutorial and learn how to ride a sel...,0,https://cryptoinsider.com/2016/03/21/read-this...,2016-03-21 12:22:53
6,"Donec quam felis, ultricies nec, pellentesque ...",Other,0,http://hashmag.mikado-themes.com/?p=217,356655,https://images.cryptocompare.com/news/cryptoin...,EN,1458562907,cryptoinsider,"{'name': 'CryptoInsider', 'lang': 'EN', 'img':...",,Read what a hundred users had to say about the...,0,https://cryptoinsider.com/2016/03/21/read-what...,2016-03-21 12:21:47
7,"Donec quam felis, ultricies nec, pellentesque ...",Other,0,http://hashmag.mikado-themes.com/?p=215,356656,https://images.cryptocompare.com/news/cryptoin...,EN,1458562850,cryptoinsider,"{'name': 'CryptoInsider', 'lang': 'EN', 'img':...",,Qualities you simply must look for when buying...,0,https://cryptoinsider.com/2016/03/21/qualities...,2016-03-21 12:20:50
8,"Donec quam felis, ultricies nec, pellentesque ...",Other,0,http://hashmag.mikado-themes.com/?p=213,356657,https://images.cryptocompare.com/news/cryptoin...,EN,1458562780,cryptoinsider,"{'name': 'CryptoInsider', 'lang': 'EN', 'img':...",,Answer these questions to find out which lapto...,0,https://cryptoinsider.com/2016/03/21/answer-th...,2016-03-21 12:19:40
9,"Donec quam felis, ultricies nec, pellentesque ...",Other,0,http://hashmag.mikado-themes.com/?p=211,356658,https://images.cryptocompare.com/news/cryptoin...,EN,1458562712,cryptoinsider,"{'name': 'CryptoInsider', 'lang': 'EN', 'img':...",,These gadgets are a necessity for every 20-som...,0,https://cryptoinsider.com/2016/03/21/these-gad...,2016-03-21 12:18:32


In [31]:
api_test = api_test.sort_values(by = 'date_time', ascending = False)

In [32]:
api_test.dtypes

body                    object
categories              object
downvotes               object
guid                    object
id                      object
imageurl                object
lang                    object
published_on             int64
source                  object
source_info             object
tags                    object
title                   object
upvotes                 object
url                     object
date_time       datetime64[ns]
dtype: object

In [33]:
api_test.title[10]

'This microscope makes it possible for anyone to see life in its smallest'

In [34]:
api_test.id.nunique() #checking number of unique articles

34868

In [35]:
#save api request to csv
#api_test.to_csv("api_data.csv")

In [36]:
api_test.to_csv("api_data_no_filters.csv")